First we extract the Data from Google Drive

In [17]:
import numpy as np
import scipy as sc
from scipy.spatial import distance
from google.colab import drive
import os
drive.mount("/content/gdrive/")
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels import regression
import statsmodels.api as sm
import math
import scipy.io as sci
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.python.keras.optimizers import *
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn import preprocessing
from sklearn import utils

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [5]:
df = pd.read_csv('/content/gdrive/My Drive/data-final.csv', sep='\t')

Then we decide which questions we will use to predict and which questions we want to predict. Code and Index of the questions are as following:

EXT1	I am the life of the party.(0)

EXT2	I don't talk a lot.(1)

EXT3	I feel comfortable around people.(2)

EXT4	I keep in the background.(3)

EXT5	I start conversations.(4)

EXT6	I have little to say.(5)

EXT7	I talk to a lot of different people at parties.(6)

EXT8	I don't like to draw attention to myself.(7)

EXT9	I don't mind being the center of attention.(8)

EXT10	I am quiet around strangers.(9)

EST1	I get stressed out easily.(10)

EST2	I am relaxed most of the time.(11)

EST3	I worry about things.(12)

EST4	I seldom feel blue.(13)

EST5	I am easily disturbed.(14)

EST6	I get upset easily.(15)

EST7	I change my mood a lot.(16)

EST8	I have frequent mood swings.(17)

EST9	I get irritated easily.(18)

EST10	I often feel blue.(19)

AGR1	I feel little concern for others.(20)

AGR2	I am interested in people.(21)

AGR3	I insult people.(22)

AGR4	I sympathize with others' feelings.(23)

AGR5	I am not interested in other people's problems.(24)

AGR6	I have a soft heart.(25)

AGR7	I am not really interested in others.(26)

AGR8	I take time out for others.(27)

AGR9	I feel others' emotions.(28)

AGR10	I make people feel at ease.(29)

CSN1	I am always prepared.(30)

CSN2	I leave my belongings around.(31)

CSN3	I pay attention to details.(32)

CSN4	I make a mess of things.(33)

CSN5	I get chores done right away.(34)

CSN6	I often forget to put things back in their proper place.(35)

CSN7	I like order.(36)

CSN8	I shirk my duties.(37)

CSN9	I follow a schedule.(38)

CSN10	I am exacting in my work.(39)

OPN1	I have a rich vocabulary.(40)

OPN2	I have difficulty understanding abstract ideas.(41)

OPN3	I have a vivid imagination.(42)

OPN4	I am not interested in abstract ideas.(43)

OPN5	I have excellent ideas.(44)

OPN6	I do not have a good imagination.(45)

OPN7	I am quick to understand things.(46)

OPN8	I use difficult words.(47)

OPN9	I spend time reflecting on things.(48)

OPN10	I am full of ideas.(49)

In [6]:
#Example set for asked questions
Asked_questions = ['EXT1','EXT2','EXT3','EXT4','EXT5','EXT6','EXT7','EXT8','EXT9','EXT10',
                    'EST1','EST2','EST3','EST4','EST5','EST6','EST7','EST8','EST9','EST10']
predicted_question = ['OPN10']
#All keyes
keyes = ['EXT1','EXT2','EXT3','EXT4','EXT5','EXT6','EXT7','EXT8','EXT9','EXT10',
                    'EST1','EST2','EST3','EST4','EST5','EST6','EST7','EST8','EST9','EST10',
                    'AGR1','AGR2','AGR3','AGR4','AGR5','AGR6','AGR7','AGR8','AGR9','AGR10',
                    'CSN1','CSN2','CSN3','CSN4','CSN5','CSN6','CSN7','CSN8','CSN9','CSN10', 
                    'OPN1','OPN2','OPN3','OPN4','OPN5','OPN6','OPN7','OPN8','OPN9','OPN10']

In [7]:
AData = df.loc[:, Asked_questions]
PData = df.loc[:, predicted_question]

In [8]:
ADataArray = AData.values
ADataArray = np.array(ADataArray)
PDataArray = PData.values
PDataArray = np.array(PDataArray)

The scale was labeled 1=Disagree, 3=Neutral, 5=Agree in the data. However,
some of the questions are not answered. So they are labeled as 0 in the data. We change them to 3 (Neutral).

In [9]:
for i in range(ADataArray.shape[0]):
  for j in range(ADataArray.shape[1]):
    if ADataArray[i][j] < 0.5:
      ADataArray[i][j] = 3
for i in range(PDataArray.shape[0]):
  for j in range(PDataArray.shape[1]):
    if PDataArray[i][j] < 0.5:
      PDataArray[i][j] = 3
print('done')

done


Then we Scale the Data, since we will use Neural Networks

In [10]:
scaler = MinMaxScaler((0, 1))
ScaledAData = scaler.fit_transform(ADataArray)
ScaledPData = scaler.fit_transform(PDataArray)

Now we split the data into train and test parts.

In [11]:
TrainValue = round(ScaledAData.shape[0]*0.8)
TrainDataX = ScaledAData[:TrainValue]
TrainDataY = ScaledPData[:TrainValue]
TrainDataY = np.reshape(TrainDataY, (TrainDataY.shape[0]))
#This one will be used in different Neural Network for classification
TrainDataY2D = np.zeros((TrainDataY.shape[0], 5))

In [12]:
print(TrainDataY.shape)

(812273,)


Now we create our models. There will 4 models.

1) Linear Regression

2) Support Vector Resgressor

3) Neural Network 1 (returns value between 0(completely disagree) and 1(agree))

4) Neural Network 2 (returns array output of size 5 (one for each class). One with biggest value will be used as prediction)

In [13]:
modelLR = LinearRegression()
modelSVR = LinearSVR(loss='squared_epsilon_insensitive', max_iter=3000)
modelNN1 = Sequential()
modelNN1.add(Dense(50, input_dim = TrainDataX.shape[1], activation='relu', use_bias=True))
modelNN1.add(Dense(40, activation='relu', use_bias=True))
modelNN1.add(Dense(20, activation='relu'))
modelNN1.add(Dense(1, activation='relu'))
modelNN2 = Sequential()
modelNN2.add(Dense(50, input_dim = TrainDataX.shape[1], activation='relu', use_bias=True))
modelNN2.add(Dense(40, activation='relu', use_bias=True))
modelNN2.add(Dense(20, activation='relu'))
modelNN2.add(Dense(5, activation='sigmoid'))

In [14]:
modelLGR = LogisticRegression(penalty='l2', max_iter=1000)

We also have to compile our Neural Networks. This is the point we decide the parameters

In [18]:
modelNN1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=False), loss='mean_squared_error')
modelNN2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=False), loss='mean_squared_error')

Data contains some Nan points. we will change them by 0.5 (neutral)

In [19]:
TrainDataY = np.nan_to_num(TrainDataY, nan=0.5)
TrainDataX = np.nan_to_num(TrainDataX, nan=0.5)

Now we create our Data set (Labels) for senond Neural network

In [20]:
for i in range(TrainDataY.shape[0]):
  TrainDataY2D[i, int(round(TrainDataY[i]*4))] = 1

In [21]:
print(TrainDataY)

[1.   0.5  0.75 ... 1.   0.5  1.  ]


Now we can train our Models.

First one is Linear Regression



In [22]:
modelLR.fit(TrainDataX, TrainDataY)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

SVR

In [23]:
modelSVR.fit(TrainDataX, TrainDataY)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=3000, random_state=None, tol=0.0001, verbose=0)

In [24]:
modelNN1.fit(TrainDataX, TrainDataY, batch_size=100, epochs=15)

Epoch 1/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.0569
Epoch 2/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0538
Epoch 3/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0536
Epoch 4/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0535
Epoch 5/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0535
Epoch 6/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0534
Epoch 7/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.0534
Epoch 8/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0534
Epoch 9/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0534
Epoch 10/15
8123/8123 [==============================] - 15s 2ms/step - loss: 0.0534
Epoch 11/15
8123/8123 [==============================] - 18s 2ms/step - loss: 0.0534
Epoch 12/15
8123/8123 [==============================] - 16s 2ms/step - lo

In [25]:
modelNN2.fit(TrainDataX, TrainDataY2D, batch_size=100, epochs=15)

Epoch 1/15
8123/8123 [==============================] - 17s 2ms/step - loss: 0.1312
Epoch 2/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1299
Epoch 3/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1296
Epoch 4/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1294
Epoch 5/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1293
Epoch 6/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1293
Epoch 7/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1292
Epoch 8/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1292
Epoch 9/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1291
Epoch 10/15
8123/8123 [==============================] - 17s 2ms/step - loss: 0.1291
Epoch 11/15
8123/8123 [==============================] - 16s 2ms/step - loss: 0.1291
Epoch 12/15
8123/8123 [==============================] - 16s 2ms/step - lo

In [26]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform((TrainDataY*4))
modelLGR.fit(TrainDataX, encoded)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Now we can test

In [27]:
TestDataX = ScaledAData[TrainValue:]
TestDataY = ScaledPData[TrainValue:]
TestDataY = np.reshape(TestDataY, (TestDataY.shape[0]))
TestDataY = np.nan_to_num(TestDataY, nan=0.5)
TestDataX = np.nan_to_num(TestDataX, nan=0.5)

Testing will be done in same order

In [28]:
predLR = modelLR.predict(TestDataX)
predSVR = modelSVR.predict(TestDataX)
predNN1 = modelNN1.predict(TestDataX)
predNN2 = modelNN2.predict(TestDataX)
predLGR = modelLGR.predict(TestDataX)

In [29]:
predLGR = predLGR / 4

In [35]:
count = 0;
for i in range(TestDataY.shape[0]):
  if(abs(TestDataY[i] - predLR[i]) < 0.251):
    count = count + 1
print(count / TestDataY.shape[0])

0.7215957216301928


In [38]:
count = 0;
for i in range(TestDataY.shape[0]):
  if(abs(TestDataY[i] - predLGR[i]) < 0.251):
    count = count + 1
print(count / TestDataY.shape[0])

0.8361829534934111


In [40]:
count = 0;
for i in range(TestDataY.shape[0]):
  if(abs(TestDataY[i] - predSVR[i]) < 0.251):
    count = count + 1
print(count / TestDataY.shape[0])

0.7243140228888845


In [41]:
count = 0;
for i in range(TestDataY.shape[0]):
  if(abs(TestDataY[i] - predNN1[i]) < 0.25):
    count = count + 1
print(count / TestDataY.shape[0])

0.7263084287036855


For now only classification Neural Network will be tested.

In [42]:
count = 0
closeCount = 0
for i in range(predNN2.shape[0]):
  maxVal = np.amax(predNN2[i])
  index = np.where(predNN2[i] == maxVal)
  if(index[0] == int(round(TestDataY[i]*4))):
    count = count + 1
  if(abs(TestDataY[i]*4 - index[0]) < 1.1):
    closeCount = closeCount + 1
print(count/TestDataY.shape[0])
print(closeCount/TestDataY.shape[0])

0.4649033821183052
0.8557330549372624
